# This jupyter file is used for LiDAR detection efficiency profiling

## 1. Import required module from OpenPCDet 

In [1]:
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
import argparse
import datetime
import glob
import os
from pathlib import Path
# from test import repeat_eval_ckpt

import spconv.pytorch as spconv
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, model_fn_decorator
from pcdet.utils import common_utils
import numpy as np
import warnings

from pcdet.models import load_data_to_gpu
import time
from pytorch_memlab import LineProfiler
import logging
import pandas as pd

warnings.filterwarnings("ignore")


## 2. Load the waymo data for profiling

**You can also re-use the dataloader defined in OpenPCDet (Recommanded)**

If you choose this option, please skip this part.

**Another option**

You can construct your own code for loading point cloud from waymo dataset. Check the code in ```../pcdet/datasets/waymo/waymo_dataset.py``` as reference. For the initial stage, loading one LiDAR scan would be enough. While later, we may need to load more to average the measurement for better accuracy.

The waymo data is located in ```../data/waymo```



In [2]:
# construct for you data loading function here

dataset_base_path = '/home/jnd/code/OpenPCDet/data/waymo'

def load_waymo_pcd():
    pass


## 3. Build the model based on cfg files

In this section, build the model following the code in ```./train.py```. The model config file in located in ```./cfgs/waymo_models```. The required configs are: pointpillar, pvrcnn, second, centerpoint. 

For profiling the efficiency of operations, you can just take the specific layer of the model for testing.

In [3]:
def load_and_build_data(model_name, batch_size):
    cfg_file = './cfgs/waymo_models/{}.yaml'.format(model_name)

    output_dir = Path('./profiling_log')
    output_dir.mkdir(exist_ok=True)
    log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=0)
    cfg_from_yaml_file(cfg_file, cfg)

    # It would take some time to initialize the dataset.

    test_set, test_loader, test_sampler = build_dataloader(
            dataset_cfg=cfg.DATA_CONFIG,
            class_names=cfg.CLASS_NAMES,
            batch_size=batch_size,
            dist=False, workers=1,
            logger=logger,
            training=False,  # I would recommand you to use a test loader
            merge_all_iters_to_one_epoch=True,
            total_epochs=1,
            seed=666
        )

    model = build_network(model_cfg=cfg.MODEL, num_class=3, dataset=test_set)
    return model, test_loader


## 4. Now load the data and feed to the network for profiling

There're two options for you to perform this task:

1. Use your own data loading function and feed the data to the network
2. Re-use the dataset to load the data. Check function **eval_one_epoch** in file ```./eval_utils/eval_utils.py``` for details.


In [4]:
# Write your code here

def inference_simulation(model_name, batch_size):
    
    model, test_loader = load_and_build_data(model_name, batch_size)
    
    model.cuda()
    model.eval()

    one_data = None
    return_data = None

    for i, batch_dict in enumerate(test_loader):
        load_data_to_gpu(batch_dict)
        one_data = batch_dict.copy()
        return_data = batch_dict.copy()
        with torch.no_grad():
            with LineProfiler(model.forward) as overall:
                start_time = time.time()
                pred_dicts, ret_dict = model(one_data)
                total_time = time.time() - start_time
        break
    return model, return_data

## 6. More experiments for specific operations

In [5]:
def load_and_build_data(model_name, batch_size):
    cfg_file = '/home/jnd/code/OpenPCDet/tools/cfgs/waymo_models/{}.yaml'.format(model_name)

    output_dir = Path('./profiling_log')
    output_dir.mkdir(exist_ok=True)
    log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=0)
    cfg_from_yaml_file(cfg_file, cfg)

    # It would take some time to initialize the dataset.

    test_set, test_loader, test_sampler = build_dataloader(
            dataset_cfg=cfg.DATA_CONFIG,
            class_names=cfg.CLASS_NAMES,
            batch_size=batch_size,
            dist=False, workers=1,
            logger=logger,
            training=False,  # I would recommand you to use a test loader
            merge_all_iters_to_one_epoch=True,
            total_epochs=1,
            seed=666
        )

    model = build_network(model_cfg=cfg.MODEL, num_class=3, dataset=test_set)
    return model, test_loader

In [6]:
model, data_loader = load_and_build_data('centerpoint', 1)

2023-01-06 14:59:16,660   INFO  Loading Waymo dataset
2023-01-06 14:59:18,118   INFO  Total skipped info 0
2023-01-06 14:59:18,118   INFO  Total samples for Waymo dataset: 39987


In [7]:
test_data = next(iter(data_loader))

(126501, 5)
polar shape is:  (126501, 5)
(128191, 5)
polar shape is:  (128191, 5)


In [8]:
model.cuda()
model.eval()
load_data_to_gpu(test_data)
result = model(test_data)

In [9]:
test_data.keys()

dict_keys(['sample_idx', 'points', 'frame_id', 'gt_boxes', 'use_lead_xyz', 'polar', 'voxels', 'voxel_coords', 'voxel_num_points', 'metadata', 'batch_size', 'voxel_features', 'encoded_spconv_tensor', 'encoded_spconv_tensor_stride', 'multi_scale_3d_features', 'multi_scale_3d_strides', 'spatial_features', 'spatial_features_stride', 'spatial_features_2d', 'final_box_dicts'])

In [10]:
pts = test_data['points']

In [11]:
pts.shape

torch.Size([126501, 6])

In [12]:
pts[:,0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [13]:
polar = test_data['polar']

In [14]:
polar.shape

torch.Size([1, 126501, 5])

In [15]:
pts.shape

torch.Size([126501, 6])

In [16]:
polar.shape

torch.Size([1, 126501, 5])